In [206]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
import inflect #numeric/cardinal

In [257]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [258]:
dataFile = gzip.open(path, 'r')

In [259]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [260]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [418]:
len(productReviews)

10261

In [286]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [287]:
productsReviewData.dtypes
len(productsReviewData)

10261

In [290]:
# reduce the data set to test text cleaning /processing before tokenizing
productsReviewData = productsReviewData.iloc[:1500, :]

In [421]:
#Domain specific handling for future use

A = productReviews[10]
A = productReviews[7]
print(A)
# search fraction 1/4 = \d+/\d+
# search out dollars amount = \$\d+.\d+
# search any digits
print(re.findall(r'\d+/\d+|\d+|\$\d+.\d+', A['reviewText']))
#print(re.findall(r'\$\d+.\d+|\d+|\d+/\d+', A['reviewText']))

#A['reviewText'].replace('$', 'dollars')

{'reviewerID': 'AJNFQI3YR6XJ5', 'asin': 'B00004Y2UT', 'reviewerName': 'Fender Guy "Rick"', 'helpful': [0, 0], 'reviewText': "I now use this cable to run from the output of my pedal chain to the input of my Fender Amp. After I bought Monster Cable to hook up my pedal board I thought I would try another one and update my guitar. I had been using a high end Planet Waves cable that I bought in the 1980's... Once I found out the input jacks on the new Monster cable didn't fit into the Fender Strat jack I was a little disappointed... I didn't return it and as stated I use it for the output on the pedal board. Save your money... I went back to my Planet Waves Cable...I payed $30.00 back in the eighties for the Planet Waves which now comes in at around $50.00. What I'm getting at is you get what you pay for. I thought Waves was a lot of money back in the day...but I haven't bought a guitar cable since this one...20 plus years and still working...Planet Waves wins.", 'overall': 3.0, 'summary': 

In [442]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

sample = A['reviewText']
sample = sample.lower()

stopWords = set(stopwords.words('english'))
review = word_tokenize(sample)
review = sent_tokenize(sample)
#len([eachWord for eachWord in review if not eachWord in stopWords])
data = {}

for eachSent in review:
    print(word_tokenize(eachSent))
    
    #print([eachWord for eachWord in eachSent if not eachWord in stopWords])

['i', 'now', 'use', 'this', 'cable', 'to', 'run', 'from', 'the', 'output', 'of', 'my', 'pedal', 'chain', 'to', 'the', 'input', 'of', 'my', 'fender', 'amp', '.']
['after', 'i', 'bought', 'monster', 'cable', 'to', 'hook', 'up', 'my', 'pedal', 'board', 'i', 'thought', 'i', 'would', 'try', 'another', 'one', 'and', 'update', 'my', 'guitar', '.']
['i', 'had', 'been', 'using', 'a', 'high', 'end', 'planet', 'waves', 'cable', 'that', 'i', 'bought', 'in', 'the', '1980', "'s", '...', 'once', 'i', 'found', 'out', 'the', 'input', 'jacks', 'on', 'the', 'new', 'monster', 'cable', 'did', "n't", 'fit', 'into', 'the', 'fender', 'strat', 'jack', 'i', 'was', 'a', 'little', 'disappointed', '...', 'i', 'did', "n't", 'return', 'it', 'and', 'as', 'stated', 'i', 'use', 'it', 'for', 'the', 'output', 'on', 'the', 'pedal', 'board', '.']
['save', 'your', 'money', '...', 'i', 'went', 'back', 'to', 'my', 'planet', 'waves', 'cable', '...', 'i', 'payed', '$', '30.00', 'back', 'in', 'the', 'eighties', 'for', 'the', 'pl

In [433]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [413]:
#handling money
#>>> a = re.findall(r"\$\d+.\d+", text2)

In [236]:
#numeric and fraction handling
#domain specific if applies to tool/measurement?
pattern = re.compile(r'^\d+/\d+$')

def evaluateFraction(data, pattern=pattern):
    try:
        re.match(pattern, data)
        return eval(data)  
    except:
        print(f"Not fraction related input {data}")
        return False      

In [417]:
p = inflect.engine() 

isNum = evaluateFraction('1/2')
print(isNum)
if not (isNum == False):
    print(p.number_to_words(isNum))

0.5
zero point five


In [293]:
productsReviewData['cleanedText'] = productsReviewData['reviewText'].astype(str)
productsReviewData['cleanedText'] = productsReviewData['cleanedText'].str.lower()

df = productsReviewData

In [415]:
pd.options.display.max_colwidth = 5000

productsReviewData.cleanedText.head(2)

0                                                                                                                                                                                                                                                                                        not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,
1    the product does exactly as it should and is quite affordable.i did not realized it was double screened until it arrived, so it was even better than i had expected.as an added bonus, one of the screens carries a small hint of the smell of an old grape candy i used to buy, so for reminiscent's sake, i cannot stop putting the pop filter next to my nose and smelling it after recording. :dif you needed a pop filter, this will work just as well a

In [121]:
sampleData = productsReviewData

In [416]:
productsReviewData.asin.nunique()

107

In [122]:
print(sampleData.asin.min())
print(sampleData.asin.max())

104800001X
B00LA96GLC


In [199]:
#pd.DataFrame(sampleData.groupby('asin')[['reviewerID'], ['helpful']])

7

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer